In [1]:
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.multiprocessing as mp
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter

rng = np.random.default_rng()

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

In [3]:
class AddGaussianNoise(nn.Module):
    def __init__(self, mean=0., std=1., *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size(), device=device) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [4]:
test_input_transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
])

train_input_transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
])

train_run_transform = transforms.Compose([
    transforms.RandomAffine(degrees=(-25,25), translate=(0.1,0.1), scale=(0.8, 1.2)),
    transforms.ElasticTransform(alpha=100.0, sigma=8.0),
    AddGaussianNoise(mean=0.0,std=0.02)
])

In [5]:
training_data = datasets.EMNIST(root="Torch MNIST", split="letters", train=True,download=True, transform=train_input_transform)
test_data = datasets.EMNIST(root="Torch MNIST", split="letters", train=False,download=True, transform=test_input_transform)

In [6]:
batch_size = int(2**9)
train_dataloader = DataLoader(training_data, batch_size = batch_size, shuffle=True, num_workers=16, pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size = batch_size, shuffle=True, num_workers=16, pin_memory=True)
batch_count = int(len(training_data)/batch_size) + 1
batch_size

512

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self,):
        super().__init__()
        self.Softmax = nn.Softmax(dim=1)
        self.Max_Pool_2D = nn.MaxPool2d(2)
        self.Activation = nn.GELU()
        
        self.conv_0 = nn.Sequential(OrderedDict([
            ("conv0", nn.Conv2d(1,16,3, padding=1)),
            ("act0", nn.GELU()),
            ("batch_norm0", nn.BatchNorm2d(16)),
            ("conv1", nn.Conv2d(16,16,3, padding=1)),
            ("act1", nn.GELU()),
            ("batch_norm1", nn.BatchNorm2d(16)),
            ("conv2", nn.Conv2d(16,16,3, padding=1)),
            ("act2", nn.GELU()),
            ("batch_norm2", nn.BatchNorm2d(16)),
            ("conv3", nn.Conv2d(16,16,3, padding=1)),
            ("act3", nn.GELU()),
            ("batch_norm3", nn.BatchNorm2d(16)),
            ("max_pool", nn.MaxPool2d(2)),
        ]))
        
        self.conv_1 = nn.Sequential(OrderedDict([
            ("conv0", nn.Conv2d(16,32,3, padding=1)),
            ("act0", nn.GELU()),
            ("batch_norm0", nn.BatchNorm2d(32)),
            ("conv1", nn.Conv2d(32,32,3, padding=1)),
            ("act1", nn.GELU()),
            ("batch_norm1", nn.BatchNorm2d(32)),
            ("conv2", nn.Conv2d(32,32,3, padding=1)),
            ("act2", nn.GELU()),
            ("batch_norm2", nn.BatchNorm2d(32)),
            ("conv3", nn.Conv2d(32,32,3, padding=1)),
            ("act3", nn.GELU()),
            ("batch_norm3", nn.BatchNorm2d(32)),
            ("max_pool", nn.MaxPool2d(2)),
        ]))
        
        self.conv_2 = nn.Sequential(OrderedDict([
            ("conv0", nn.Conv2d(32,64,3, padding=1)),
            ("act0", nn.GELU()),
            ("batch_norm0", nn.BatchNorm2d(64)),
            ("conv1", nn.Conv2d(64,64,3, padding=1)),
            ("act1", nn.GELU()),
            ("batch_norm1", nn.BatchNorm2d(64)),
            ("conv2", nn.Conv2d(64,64,3, padding=1)),
            ("act2", nn.GELU()),
            ("batch_norm2", nn.BatchNorm2d(64)),
            ("conv2", nn.Conv2d(64,64,3, padding=1)),
            ("act2", nn.GELU()),
            ("batch_norm2", nn.BatchNorm2d(64)),
            ("max_pool", nn.MaxPool2d(2)),
        ]))
        
        self.conv_3 = nn.Sequential(OrderedDict([
            ("conv0", nn.Conv2d(64,128,3, padding=1)),
            ("act0", nn.GELU()),
            ("batch_norm0", nn.BatchNorm2d(128)),
            ("conv1", nn.Conv2d(128,128,3, padding=1)),
            ("act1", nn.GELU()),
            ("batch_norm1", nn.BatchNorm2d(128)),
            ("conv2", nn.Conv2d(128,128,3, padding=1)),
            ("act2", nn.GELU()),
            ("batch_norm2", nn.BatchNorm2d(128)),
            ("conv3", nn.Conv2d(128,128,3, padding=1)),
            ("act3", nn.GELU()),
            ("batch_norm3", nn.BatchNorm2d(128)),
            ("max_pool", nn.MaxPool2d(2)),
        ]))
        
        self.linear_stack = nn.Sequential(OrderedDict([
            ("flatten", nn.Flatten()),
            ("dropout1", nn.Dropout(0.3)),
            ("linear1", nn.Linear(4*4*128, 2048)),
            ("act1", nn.GELU()),
            ("batch_norm1", nn.BatchNorm1d(2048)),
            ("dropout2", nn.Dropout(0.3)),
            ("linear2", nn.Linear(2048, 2048)),
            ("act2", nn.GELU()),
            ("batch_norm2", nn.BatchNorm1d(2048)),
            ("linear3", nn.Linear(2048,37)),
        ]))

    def forward(self, x):
        if self.training:
            x = train_run_transform(x)
        
        x = self.conv_0(x)
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.conv_3(x)
        x = self.linear_stack(x)
        
        if self.training:
            return x
        else:
            return self.Softmax(x)

In [8]:
model = NeuralNetwork().to(device)

summary(model, input_size=(batch_size, 1, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [512, 37]                 --
├─Sequential: 1-1                        [512, 16, 32, 32]         --
│    └─Conv2d: 2-1                       [512, 16, 64, 64]         160
│    └─GELU: 2-2                         [512, 16, 64, 64]         --
│    └─BatchNorm2d: 2-3                  [512, 16, 64, 64]         32
│    └─Conv2d: 2-4                       [512, 16, 64, 64]         2,320
│    └─GELU: 2-5                         [512, 16, 64, 64]         --
│    └─BatchNorm2d: 2-6                  [512, 16, 64, 64]         32
│    └─Conv2d: 2-7                       [512, 16, 64, 64]         2,320
│    └─GELU: 2-8                         [512, 16, 64, 64]         --
│    └─BatchNorm2d: 2-9                  [512, 16, 64, 64]         32
│    └─Conv2d: 2-10                      [512, 16, 64, 64]         2,320
│    └─GELU: 2-11                        [512, 16, 64, 64]         --
│    

In [9]:
run_comment = input("Run Comment: ")
writer = SummaryWriter(comment=" " + run_comment)
writer.add_scalar("Loss/Train", 1.0, 0)
writer.add_scalar("Accuracy/Test", 0, 0)
writer.add_scalar("Accuracy/Train", 0, 0)
epoch_count = 0

In [ ]:
loss_fn = nn.CrossEntropyLoss()
Optimiser = optim.AdamW(model.parameters(), lr=1e-2, eps=3e-5, weight_decay=1e-6)
model.train(True)
accuracy_test_freq = 1

for epoch in range(0,1000):
    
    running_loss = 0.0
    batch_loss = 0.0
    total = 0
    correct = 0
    
    for batch_num, batch_data in enumerate(train_dataloader):
        #print("Epoch {0} Batch {1}".format(epoch+1, batch_num+1))
        Optimiser.zero_grad()
        
        data, labels = batch_data
        #labels = nn.functional.one_hot(labels, num_classes = 47).float()
        data = data.to(device)
        labels = labels.to(device)
        
        
        prediction = model(data)
        _, predicted = torch.max(prediction, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = loss_fn(prediction,labels)
        
        loss.backward()
        
        running_loss += loss.item()
        batch_loss += loss.item()
        log_num = int(batch_count/16)
        if log_num == 1:
            print(f'[{epoch + 1}, {batch_num + 1:5d}] loss: {batch_loss:.3f}')
            writer.add_scalar("Loss/Train", batch_loss / log_num, epoch_count * batch_count + batch_num)
            writer.add_scalar("Accuracy/Train", (correct/total) * 100, epoch_count * batch_count + batch_num)
            batch_loss = 0.0
            total = 0
            correct = 0
        else:
            if batch_num % log_num == (log_num-1):
                print(f'[{epoch + 1}, {batch_num + 1:5d}] loss: {batch_loss / log_num:.3f}')
                writer.add_scalar("Loss/Train", batch_loss / log_num, epoch_count * batch_count + batch_num)
                writer.add_scalar("Accuracy/Train", (correct/total) * 100, epoch_count * batch_count + batch_num )
                batch_loss = 0.0
                total = 0
                correct = 0
        Optimiser.step()
        
        running_loss = 0.0
        total = 0
        correct = 0
        
    if (epoch+1) % accuracy_test_freq == 0:
        with torch.no_grad():
            model.train(False)
            for loader_data in test_dataloader:
                data, labels = loader_data
                data = data.to(device)
                labels = labels.to(device)
                output = model(data)
                _, predicted = torch.max(output, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            model.train(True)
        writer.add_scalar("Accuracy/Test", (correct/total) * 100, epoch_count + 1)
        
    epoch_count += 1
    

[1,    15] loss: 6.642
[1,    30] loss: 3.195
[1,    45] loss: 2.127
[1,    60] loss: 1.728
[1,    75] loss: 1.454
[1,    90] loss: 1.420
[1,   105] loss: 1.154
[1,   120] loss: 0.969
[1,   135] loss: 0.930
[1,   150] loss: 1.114
[1,   165] loss: 0.873
[1,   180] loss: 0.970
[1,   195] loss: 1.042
[1,   210] loss: 1.019
[1,   225] loss: 1.097
[1,   240] loss: 0.852
[2,    15] loss: 0.740
[2,    30] loss: 0.773
[2,    45] loss: 1.049
[2,    60] loss: 0.799
[2,    75] loss: 0.588
[2,    90] loss: 0.575
[2,   105] loss: 0.586
[2,   120] loss: 0.542
[2,   135] loss: 0.486
[2,   150] loss: 0.477
[2,   165] loss: 0.458
[2,   180] loss: 0.487
[2,   195] loss: 0.489
[2,   210] loss: 0.845
[2,   225] loss: 0.766
[2,   240] loss: 0.532
[3,    15] loss: 0.589
[3,    30] loss: 0.459
[3,    45] loss: 0.456
[3,    60] loss: 0.380
[3,    75] loss: 0.426
[3,    90] loss: 0.393
[3,   105] loss: 0.406
[3,   120] loss: 0.366
[3,   135] loss: 0.379
[3,   150] loss: 0.333
[3,   165] loss: 0.355
[3,   180] 

In [ ]:
total = 0
correct = 0
model.eval()

with torch.no_grad():
    for loader_data in test_dataloader:
        data, labels = loader_data
        data = data.to(device)
        labels = labels.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print((correct/total) * 100)